In [1]:
import pandas as pd

In [2]:
import re
import string

In [3]:
from nltk.corpus import stopwords
import nltk

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
from sklearn.decomposition import NMF

In [9]:
df_union = pd.read_csv('df_union_all.csv')  

/Users/albinasitdikova/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
df_union

## Text preprocessing

In [1]:
df_union.columns

NameError: name 'df_union' is not defined

Delete reviews that contains more non-english words than english, because a lot of french words defined as english

In [ ]:
words = set(nltk.corpus.words.words())

In [ ]:

words = set(nltk.corpus.words.words())

sent = "Io andiamo to the beach with my amico."
" ".join(w for w in nltk.wordpunct_tokenize(sent) \
         if w.lower() in words or not w.isalpha())
# 'Io to the beach with my'

In [ ]:
df_union.locality.value_counts()

In [ ]:
# Compound Term Extraction
repl = lambda x: x.replace('new york', 'new_york').replace('nyc', 'new_york').replace('times square', 'times_square').replace('new york city', 'new_york')#.replace('nyc', 'new_york').replace('nyc', 'new_york').replace('nyc', 'new_york').replace('nyc', 'new_york')

# Text preprocessing steps - remove numbers, captial letters and punctuation
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

words = set(nltk.corpus.words.words())
english_only = lambda x: " ".join(w for w in nltk.wordpunct_tokenize(x) \
         if w.lower() in words or not w.isalpha())

df_union['text'] = df_union.text.map(alphanumeric).map(punc_lower).map(repl)#.map(english_only) #.map(repl)
df_union['title'] = df_union.title.map(alphanumeric).map(punc_lower).map(repl)#.map(english_only) #.map(repl)
df_union.head()

In [ ]:
#instead all non-English review will type 0, and return only English words
def is_english(row):
    
    len_review = len(row)
    english_review = " ".join(w for w in nltk.wordpunct_tokenize(row) \
         if w.lower() in words or not w.isalpha())
    
    len_english = len(english_review)
    
    if len_review > len_english * 2:
        return 0
    else:
        return english_review

In [ ]:
df_union.text = df_union.text.apply(is_english)

In [ ]:
df_english = df_union.copy()

In [ ]:
df_english.drop(df_english.loc[df_english['text']==0].index, inplace=True)

In [ ]:
df_english 

In [ ]:
# removing stop words from the text
stop = stopwords.words('english')
stop += ['.', ',', '(', ')', "'", '"']
stop += ['hotel', 'stay']
stop = set(stop)

In [ ]:
df_union

#### Delete reviews where author_id or id id None

In [ ]:
df_english.dropna(subset = ["offering_id"], inplace=True)

In [ ]:
df_english.dropna(subset = ["id"], inplace=True)

In [ ]:
df_english.reset_index(drop=True,inplace=True)

In [ ]:
df_english

## Document-Term Matrix

In [ ]:
# The first document-term matrix has default Count Vectorizer values - counts of unigrams

vectorizer = CountVectorizer(stop_words = stop)

doc_word = vectorizer.fit_transform(df_english.text)

pd.DataFrame(doc_word.toarray(), columns=vectorizer.get_feature_names()).head()

## Topic modeling

### Non-Negative Matrix Factorization (NMF)

In [ ]:
nmf_model = NMF(10)
doc_topic_nmf = nmf_model.fit_transform(doc_word)
doc_topic_nmf.shape

In [ ]:
topic_word_nmf = nmf_model.components_
topic_word_nmf.shape

In [ ]:
words_nmf = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-8:-1]
topic_words_nmf = [[words_nmf[e] for e in l] for l in t]
topic_words_nmf

### Try using TF-IDF instead of Count Vectorizer

In [ ]:
# Create TF-IDF versions of the Count Vectorizers created earlier in the exercise
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(stop_words=stop)

In [ ]:
doc_word_tfidf = tfidf.fit_transform(df_english.text)              

In [ ]:
pd.DataFrame(doc_word_tfidf.toarray(), columns=tfidf.get_feature_names()).head()

#### NMF - tf-ifd

In [ ]:
nmf_model_2 = NMF(10)
doc_topic_nmf_2 = nmf_model_2.fit_transform(doc_word_tfidf)
doc_topic_nmf_2.shape

In [ ]:
topic_word_nmf_2 = nmf_model_2.components_
topic_word_nmf_2.shape

In [ ]:
words_2 = tfidf.get_feature_names()
t_2 = nmf_model_2.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words_nmf_2 = [[words_2[e] for e in l] for l in t_2]
topic_words_nmf_2

In [ ]:
doc_topic_nmf_2

In [ ]:
H = pd.DataFrame(doc_topic_nmf_2.round(5),
                 index = df_english.text
                )
H

### !!! try different topic modeling on tf-idf

### LSA

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
lsa = TruncatedSVD(10)
doc_topic_lsa = lsa.fit_transform(doc_word_tfidf)
lsa.explained_variance_ratio_

In [ ]:
doc_topic_lsa

In [ ]:
Vt = pd.DataFrame(doc_topic_lsa.round(5),
             index = df_english.text)
Vt

In [ ]:
words_lsa = tfidf.get_feature_names()
t_lsa = lsa.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words_lsa = [[words_lsa[e] for e in l] for l in t_lsa]
topic_words_lsa

### LDA

### Text Recommendation through Matrix Factorization

In [ ]:
from sklearn.metrics import pairwise_distances

In [ ]:
pairwise_distances(doc_topic_nmf_2[4].reshape(1,-1),doc_topic_nmf_2,metric='cosine')

In [ ]:
pairwise_distances(doc_topic_nmf_2[4].reshape(1,-1),doc_topic_nmf_2,metric='cosine').argsort()

##### You can now take any text and make a recommendation for it:

In [ ]:
t = ['basketball is the best']

In [ ]:
vt = vectorizer.transform(t)

In [ ]:
tt = nmf_model.transform(vt)

In [ ]:
pairwise_distances(tt,doc_topic_nmf_2,metric='cosine').argsort()

In [ ]:
df_english.text[1272]

Simple recommender: list of similar hotels

In [ ]:
from collections import defaultdict

In [ ]:
user_movie_map = defaultdict(list)
hotel_user_map = defaultdict(list)

In [ ]:
user_movie_map

In [ ]:
df_english.columns

In [ ]:
df_english

In [ ]:
len(df_english)

In [ ]:
df_english.loc[0, 'author_id']

In [ ]:
user_movie_map

In [ ]:
df_english.shape[0]

In [ ]:
df_english.head(10)

In [ ]:
df_english.author_id[9]

In [ ]:
for i in range(0, df_english.shape[0]):
#     print(df_english.author_id[i])
#     print(df_english.id[i])
#     print(i)""
    user_movie_map[df_english.author_username[i]].append(df_english.id[i])
    
    hotel_user_map[df_english.id[i]].append(df_english.author_username[i])

In [ ]:
user_movie_map

In [ ]:
def get_similar_hotels(user_movie_map,hotel_user_map,m):
    biglist = []
    for u in hotel_user_map[m]: # get all users that liked this movie
        biglist.extend(user_movie_map[u]) # find all other movies those users liked and add to biglist
    return Counter(biglist).most_common(4)[1:] # use counter to 'count' the other movies that show up most common

In [ ]:
from collections import Counter

In [ ]:
get_similar_hotels(user_movie_map,hotel_user_map,133618731.0)

In [ ]:
hotel_user_map

## Surprise recommender

In [3]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy
from surprise import Reader
from surprise import BaselineOnly

In [10]:
df_union_all = pd.read_csv('df_union_all.csv')  

In [8]:
df_union_all.head()

,title,text,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile,service,cleanliness,...,region_id,url,phone,details,type,name,region,street-address,postal-code,locality
0,"“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,December 2012,93338,12.0,2012-12-17,8C0B42FF3C0FA366A21CFD785302A032,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
1,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...",December 2012,93338,NaN,2012-12-17,E3C85CA9DBBBC77E0DB534ABE93E4713,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
2,“Excellent location”,Loved the hotel. Great location - only 2 block...,December 2012,93338,1.0,2012-12-17,035080D10D186778123E6A58C11E0608,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
3,“All-round fantastic NYC hotel”,Our first stay on the upper west side and can'...,December 2012,93338,NaN,2012-12-17,412A714D6BF1607A01CC828645DD70AE,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
4,“Great hotel in nice area”,"Great room, very big with huge bed! Great loca...",December 2012,93338,3.0,2012-12-17,72B051F423790BE5F0EF6A9FD23EB77E,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City


In [9]:
df_union_all.columns

Index(['title', 'text', 'date_stayed', 'offering_id', 'num_helpful_votes',
       'date', 'id', 'via_mobile', 'service', 'cleanliness', 'overall',
       'value', 'location', 'sleep_quality', 'rooms', 'check_in_front_desk',
       'business_service_(e_g_internet_access)', 'username', 'num_cities',
       'num_reviews', 'num_type_reviews', 'hotel_class', 'region_id', 'url',
       'phone', 'details', 'type', 'name', 'region', 'street-address',
       'postal-code', 'locality'],
      dtype='object')

In [11]:
df_surprise = df_union_all.loc[:,['id', 'offering_id', 'overall']]

In [12]:
df_surprise

5    322771
4    265623
3    112218
2     53280
1     47302
Name: overall, dtype: int64

In [12]:
#df_surprise['overall'] = pd.to_numeric(df_surprise.overall)

In [13]:
#use only reviews that > 5


In [6]:
#count reviews for each user
reviews_count_byuser = df_surprise.groupby(['id'])['overall'].count().sort_values(ascending=False)
reviews_count_byuser

id
6562BBD4EA770FE84E579622F68FA181    63
869E5453ED1ECD12EC2E210BEB6B922E    54
6C62B19C4DB8C600576B763C68AF0759    52
85223AA53CB0DE6A6EB7B3C1E99981B1    52
7BC0AF07CC240F2F614A865BBE21B5AA    47
                                    ..
A0C0ABF1FDB3C9DE56BA848DA14F6348     1
A0C0D203DEB1B61000BBF9259A4B5A22     1
A0C0DCCEC004AABAEFB5CD88A6ED4BDA     1
A0C0E290EABB5C946BF70442D7CDE19A     1
7FC56E9337C70B8D630FCFB59E9A988E     1
Name: overall, Length: 576685, dtype: int64

In [40]:
user_more_reviews = reviews_count_byuser[reviews_count_byuser>5]
user_more_reviews.index

Index(['6562BBD4EA770FE84E579622F68FA181', '869E5453ED1ECD12EC2E210BEB6B922E',
       '6C62B19C4DB8C600576B763C68AF0759', '85223AA53CB0DE6A6EB7B3C1E99981B1',
       '7BC0AF07CC240F2F614A865BBE21B5AA', 'F73CC60121EDAB8E6B1637FDE6A09177',
       '17EEAF9A079A2B75E9616F7864F7CA3E', '5A259EBC8C1AEBD6F20D4C48490255FF',
       '106952AB894E1236A6094B030DD73C8F', '5DFE96EC85C67F248DEFFA8B84891A6A',
       ...
       '00BBAC3339F576E164FF9F627489481C', 'FD2BB013EE49D8432414008F8B89C482',
       'FEF59A627074937521F622D5F65794AF', '01ECD34DFB3A71980CFE05D453CB2156',
       'FD2680BF42CF51B2B13A136983B804C4', 'FDB8E88150901C82631CD5548B1A2BB8',
       '027E1165FCD77427C33454661AF677CA', 'FCE4AD28BE08AF16A5B8E91D4D3C4F40',
       'FD502B68BDE687E6E3F59E0578A8B615', 'FFE779A7913FECF3B0476DEB287ADCA8'],
      dtype='object', name='id', length=7252)

In [41]:
df_surprise.id.isin(user_more_reviews.index)

0         False
5         False
10        False
13        False
18        False
          ...  
801189    False
801190    False
801191    False
801192    False
801193    False
Name: id, Length: 343996, dtype: bool

In [42]:
#df_surprise['user'] = df_surprise.id.isin(user_more_reviews.index)

In [43]:
df_surprise = df_surprise[df_surprise.id.isin(user_more_reviews.index)]

In [44]:
df_surprise 

,id,offering_id,overall
35,1005ABAA83773778AB2A4BAAFE443901,93338,5
39,FD711E1ED5A2DA3F69425CDD77A80E5A,93338,5
63,A12D74E5CFC1564773D7C48B490DBE54,93338,5
119,3D693454C428B1D6E5458ACC210F9695,93338,5
120,771E3C17495EE9D080AF87EB2504161F,93338,4
...,...,...,...
801149,0FEEEDDB251FEF3713076F4CBFB08CF7,84117,5
801153,1437F74083AF6DEE8DAAC603F436EDA0,84117,5
801161,657EB42F3264EA15A287F416268A8A68,84117,5
801165,40923C2E4B6283C7450E045801624447,84117,4


In [45]:
#hotel has more than 5 review

In [46]:
reviews_count_by_hotel = df_surprise.groupby(['offering_id'])['overall'].count().sort_values(ascending=False)
reviews_count_by_hotel

offering_id
87617      297
111501     284
93618      244
611947     244
93507      240
          ... 
1235341      1
98428        1
1229785      1
223136       1
3523356      1
Name: overall, Length: 3020, dtype: int64

In [47]:
hotel_more_reviews = reviews_count_by_hotel[reviews_count_by_hotel>5]
hotel_more_reviews.index

Int64Index([ 87617, 111501,  93618, 611947,  93507, 249712,  93450,  81295,
             81461, 100570,
            ...
            729371,  80733,  91484,  81625,  81511, 109201, 223669, 278567,
            247154,  73905],
           dtype='int64', name='offering_id', length=1775)

In [48]:
df_surprise.offering_id.isin(hotel_more_reviews.index)

35        True
39        True
63        True
119       True
120       True
          ... 
801149    True
801153    True
801161    True
801165    True
801167    True
Name: offering_id, Length: 60976, dtype: bool

In [49]:
df_surprise = df_surprise[df_surprise.offering_id.isin(hotel_more_reviews.index)]
df_surprise

,id,offering_id,overall
35,1005ABAA83773778AB2A4BAAFE443901,93338,5
39,FD711E1ED5A2DA3F69425CDD77A80E5A,93338,5
63,A12D74E5CFC1564773D7C48B490DBE54,93338,5
119,3D693454C428B1D6E5458ACC210F9695,93338,5
120,771E3C17495EE9D080AF87EB2504161F,93338,4
...,...,...,...
801149,0FEEEDDB251FEF3713076F4CBFB08CF7,84117,5
801153,1437F74083AF6DEE8DAAC603F436EDA0,84117,5
801161,657EB42F3264EA15A287F416268A8A68,84117,5
801165,40923C2E4B6283C7450E045801624447,84117,4


# For surprise only review that has more than 5 review for each hotel and more than 5 review from each user?

In [13]:
#df_surprise.to_csv('df_surprise_all.csv', index=False, header=False)

In [50]:
df_surprise.to_csv('df_surprise.csv', index=False, header=False)

In [51]:
file_path = 'df_surprise.csv'

In [52]:
#file_path = os.path.expanduser('df_surprise.csv')

In [53]:
# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
reader = Reader(line_format='user item rating', sep=',')

In [54]:
data = Dataset.load_from_file(file_path, reader=reader)

In [55]:
# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(BaselineOnly(), data, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8917  0.8717  0.8867  0.8793  0.8842  0.8827  0.0068  
MAE (testset)     0.6929  0.6757  0.6833  0.6824  0.6873  0.6843  0.0057  
Fit time          0.25    0.49    0.37    0.27    0.26    0.33    0.09    
Test time         0.12    0.12    0.20    0.11    0.13    0.14    0.04    


{'test_rmse': array([0.89166806, 0.87167649, 0.88670759, 0.87930057, 0.88419753]),
 'test_mae': array([0.69294484, 0.67568599, 0.68327726, 0.68236237, 0.68732892]),
 'fit_time': (0.25406384468078613,
  0.48786091804504395,
  0.3713071346282959,
  0.26659703254699707,
  0.2614412307739258),
 'test_time': (0.11557316780090332,
  0.11806797981262207,
  0.20440292358398438,
  0.10858607292175293,
  0.1284470558166504)}

In [20]:
# We'll use the famous SVD algorithm.
algo = SVD(verbose=True)

# you can also build KNNBasic and other types of models

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1, verbose=True)

# ml-100k dataset: this takes around .5 minute
# jester dataset: this takes around 10 minutes

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9518  0.9471  0.9539  0.9452  0.9522  0.9501  0.0033  
MAE (testset)     0.7368  0.7353  0.7383  0.7324  0.7365  0.7358  0.0020  
Fit time          47.53   47.42   47.55   47.10   20.45   42.01   10.78   
Test time         1.32    1.38    1.42    1.38    0.70    1.24    0.27    


{'test_rmse': array([0.95180342, 0.94713722, 0.95392451, 0.94522223, 0.95223164]),
 'test_mae': array([0.73675807, 0.73528222, 0.73828133, 0.73238672, 0.73652023]),
 'fit_time': (47.53168606758118,
  47.4227991104126,
  47.55194711685181,
  47.09607672691345,
  20.445478916168213),
 'test_time': (1.3196218013763428,
  1.377467155456543,
  1.4233999252319336,
  1.3836021423339844,
  0.7045061588287354)}

In [21]:
# let's do train-test-split, where test set is 25% of the ratings
trainset, testset = train_test_split(data, test_size=.25)

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# you can also use this one-liner: `predictions = algo.fit(trainset).test(testset)`

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [22]:
# compute RMSE
accuracy.rmse(predictions)

RMSE: 0.9540


0.9539688845105767

In [23]:
# get a prediction for specific users and items.
uid = '8C0B42FF3C0FA366A21CFD785302A032'
iid = '84117'

pred = algo.predict(uid, iid, verbose=True)

user: 8C0B42FF3C0FA366A21CFD785302A032 item: 84117      r_ui = None   est = 4.90   {'was_impossible': False}


In [24]:
df_surprise[df_surprise.id=='8C0B42FF3C0FA366A21CFD785302A032']

,id,offering_id,overall
0,8C0B42FF3C0FA366A21CFD785302A032,93338,5
210114,8C0B42FF3C0FA366A21CFD785302A032,223024,5
285914,8C0B42FF3C0FA366A21CFD785302A032,111486,5


In [25]:
df_surprise.offering_id.value_counts()

611947     1360
93618      1320
93507      1273
111501     1234
87617      1221
           ... 
1199530       2
88429         2
239891        2
656584        2
105191        2
Name: offering_id, Length: 3455, dtype: int64

In [26]:
df_surprise[df_surprise.offering_id==3337599]

,id,offering_id,overall


In [27]:
df_surprise[df_surprise.id=='3FE29CCADCF6AB444B131848DE3E7759']

,id,offering_id,overall


In [28]:
# get a prediction for specific users and items.
uid = '8C0B42FF3C0FA366A21CFD785302A032'
iid = 3337599

pred = algo.predict(uid, iid, verbose=True)

user: 8C0B42FF3C0FA366A21CFD785302A032 item: 3337599    r_ui = None   est = 4.07   {'was_impossible': False}


In [29]:
uid = '22'
iid = 22

pred = algo.predict(uid, iid, verbose=True)

user: 22         item: 22         r_ui = None   est = 3.95   {'was_impossible': False}


### WHY imposibble FALSE??? If new user and new hotel?

In [30]:
from collections import defaultdict

from surprise import SVD
from surprise import Dataset

In [56]:
# First train an SVD algorithm on the movielens dataset.
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

In [ ]:
# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [ ]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
top_n = get_top_n(predictions, n=10)

In [38]:
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

NameError: name 'top_n' is not defined

In [37]:
top_n

NameError: name 'top_n' is not defined

In [34]:
df_union_all

,title,text,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile,service,cleanliness,...,region_id,url,phone,details,type,name,region,street-address,postal-code,locality
0,"“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,December 2012,93338,12.0,2012-12-17,8C0B42FF3C0FA366A21CFD785302A032,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
1,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...",December 2012,93338,NaN,2012-12-17,E3C85CA9DBBBC77E0DB534ABE93E4713,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
2,“Excellent location”,Loved the hotel. Great location - only 2 block...,December 2012,93338,1.0,2012-12-17,035080D10D186778123E6A58C11E0608,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
3,“All-round fantastic NYC hotel”,Our first stay on the upper west side and can'...,December 2012,93338,NaN,2012-12-17,412A714D6BF1607A01CC828645DD70AE,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
4,“Great hotel in nice area”,"Great room, very big with huge bed! Great loca...",December 2012,93338,3.0,2012-12-17,72B051F423790BE5F0EF6A9FD23EB77E,False,5.0,5.0,...,60763,http://www.tripadvisor.com/Hotel_Review-g60763...,NaN,NaN,hotel,Hotel Beacon,NY,2130 Broadway at 75th Street,10023,New York City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801189,“hotel mytique”,"Extrêmement bien situé, en face de la maison b...",May 2009,84117,11.0,2010-03-01,090EA46E480A3E1A3D804B7DBEA9C21E,False,NaN,NaN,...,28970,http://www.tripadvisor.com/Hotel_Review-g28970...,NaN,NaN,hotel,The Hay-Adams,DC,16th & H Sts. NW,20006,Washington DC
801190,“Die gute alte Hotelschule”,"Ich war in vielen alten Leadinghotels,Hay Adam...",September 2009,84117,1.0,2009-10-03,6E1191432CA21D10DE10A75F00DD50D9,False,5.0,5.0,...,28970,http://www.tripadvisor.com/Hotel_Review-g28970...,NaN,NaN,hotel,The Hay-Adams,DC,16th & H Sts. NW,20006,Washington DC
801191,“Le charme d'un hôtel vieillot”,"5 nuits dans cet hôtel merveilleusement situé,...",February 2009,84117,3.0,2009-03-14,A6F3EB0FDFB002EB4D47554A6D0AFCB1,False,4.0,5.0,...,28970,http://www.tripadvisor.com/Hotel_Review-g28970...,NaN,NaN,hotel,The Hay-Adams,DC,16th & H Sts. NW,20006,Washington DC
801192,“Excellent ! Le meilleur emplacement pour visi...,Un merveilleux boutique hôtel au décor dans le...,August 2008,84117,11.0,2008-09-10,53E01D7C826C4091AAC83A38B6E88B06,False,5.0,5.0,...,28970,http://www.tripadvisor.com/Hotel_Review-g28970...,NaN,NaN,hotel,The Hay-Adams,DC,16th & H Sts. NW,20006,Washington DC


## k nearest neighbors of item

In [ ]:
import io  # needed because of weird encoding of u.item file

from surprise import KNNBaseline
from surprise import Dataset
from surprise import get_dataset_dir

In [ ]:
def read_item_names():
    """Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    """

    file_name = file_path
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid

In [ ]:
# First, train the algortihm to compute the similarities between items
trainset = data.build_full_trainset()
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBaseline(sim_options=sim_options)
algo.fit(trainset)

In [ ]:
# Read the mappings raw id <-> movie name
rid_to_name, name_to_rid = read_item_names()

# Retrieve inner id of the hotel_id 84117
toy_story_raw_id = name_to_rid['84117']
toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)

# Retrieve inner ids of the nearest neighbors of Toy Story.
toy_story_neighbors = algo.get_neighbors(toy_story_inner_id, k=10)

# Convert inner ids of the neighbors into names.
toy_story_neighbors = (algo.trainset.to_raw_iid(inner_id)
                       for inner_id in toy_story_neighbors)
toy_story_neighbors = (rid_to_name[rid]
                       for rid in toy_story_neighbors)

In [ ]:
print()
print('The 10 nearest neighbors of Toy Story are:')
for movie in toy_story_neighbors:
    print(movie)

In [2]:
df_surprise_all = pd.read_csv('df_surprise_all.csv')  

In [8]:
df_surprise_all.columns

Index(['8C0B42FF3C0FA366A21CFD785302A032', '93338', '5'], dtype='object')